In [2]:
import numpy as np
import numpy.testing as npt

In [3]:
im = np.zeros((608,608))
im[399,399]=1
im[607,607]=1
im[607,0]=1
im[0,607]=1

In [4]:
# Extract patches from a given image
def img_crop_(im, h, w):
    list_patches = []
    imgwidth = im.shape[0]
    imgheight = im.shape[1]
    is_2d = len(im.shape) < 3
    
    for i in range(0, imgheight, h):
        
        i0 = i
        
        for j in range(0, imgwidth, w):
            
            j0 = j
            
            j_plus_w = j+w-1
            i_plus_h = i+h-1
            
            if i_plus_h >= imgheight:
                i_plus_h = imgheight-1
                i0= imgheight - h
                        
            if j_plus_w >= imgwidth:
                j_plus_w = imgwidth-1
                j0 = imgwidth - w
                
            print("i0 = ", i0, "i_plus_h = ", i_plus_h)
            print("j0 = ", j0, "j_plus_w = ", j_plus_w)
            
            if is_2d:
                im_patch = im[i0:i_plus_h+1, j0:j_plus_w+1]
            else:
                im_patch = im[i0:i_plus_h+1, j0:j_plus_w+1, :]
                
            print("im_patch : ", im_patch.shape)
            list_patches.append(im_patch)

    
    return list_patches

list_patches = img_crop_(im,400,400)

i0 =  0 i_plus_h =  399
j0 =  0 j_plus_w =  399
im_patch :  (400, 400)
i0 =  0 i_plus_h =  399
j0 =  208 j_plus_w =  607
im_patch :  (400, 400)
i0 =  208 i_plus_h =  607
j0 =  0 j_plus_w =  399
im_patch :  (400, 400)
i0 =  208 i_plus_h =  607
j0 =  208 j_plus_w =  607
im_patch :  (400, 400)


In [5]:
npt.assert_equal(list_patches[0][399,399],1)
npt.assert_equal(list_patches[1][399,399-208],1)
npt.assert_equal(list_patches[2][399-208,399],1)
npt.assert_equal(list_patches[3][399-208,399-208],1)
npt.assert_equal(list_patches[1][0,399],1)
npt.assert_equal(list_patches[2][399,0],1)
npt.assert_equal(list_patches[3][399,399],1)

In [6]:
def build_from_patches(list_patches, h, w):
    patch_width = list_patches[0].shape[0]
    patch_height = list_patches[0].shape[1]
    is_2d = len(list_patches[0].shape) < 3
    rebuild_img = np.zeros((h, w))
    
    npt.assert_equal(is_2d == True, True)
    npt.assert_equal(len(list_patches) == 4, True)
    
    for i in range(0, patch_height):
        for j in range(0, patch_width):
            rebuild_img[i,j] = list_patches[0][i,j]

    width_begin = patch_width-(w%patch_width)
    for i in range(0, patch_height):
        for j in range(width_begin, patch_width):
            rebuild_img[i,patch_width+j-width_begin] = list_patches[1][i,j]

    height_begin = patch_height-(h%patch_height)
    for i in range(height_begin, patch_height):
        for j in range(0, patch_width):
            rebuild_img[patch_height+i-height_begin,j] = list_patches[2][i,j]

    for i in range(patch_height-(h%patch_height), patch_height):
        for j in range(0, patch_width):
            rebuild_img[patch_height+i-height_begin,patch_width+j-width_begin] = list_patches[3][i,j]

    return rebuild_img
                
rebuild_img = build_from_patches(list_patches, 608, 608)

npt.assert_equal(rebuild_img[399,399],1)
npt.assert_equal(rebuild_img[0,607],1)
npt.assert_equal(rebuild_img[607,0],1)
npt.assert_equal(rebuild_img[607,607],1)


In [11]:
def build_mask_taking_max(list_patches, h, w):
    patch_width = list_patches[0].shape[0]
    patch_height = list_patches[0].shape[1]
    is_2d = len(list_patches[0].shape) < 3
    rebuild_img = np.zeros((h, w))
    
    npt.assert_equal(is_2d == True, True)
    npt.assert_equal(len(list_patches) == 4, True)
    
    for i in range(0, patch_height):
        for j in range(0, patch_width):
            val = list_patches[0][i,j]
            if val > 0:
                rebuild_img[i,j] = val

    width_begin = patch_width-(w%patch_width)
    for i in range(0, patch_height):
        for j in range(0, patch_width):
            val = list_patches[1][i,j]
            if val > 0:
                rebuild_img[i,patch_width+j-width_begin] = val

    height_begin = patch_height-(h%patch_height)
    for i in range(0, patch_height):
        for j in range(0, patch_width):
            val = list_patches[2][i,j]
            if val > 0:
                rebuild_img[patch_height+i-height_begin,j] = val

    for i in range(0, patch_height):
        for j in range(0, patch_width):
            val = list_patches[3][i,j]
            if val > 0:
                rebuild_img[patch_height+i-height_begin,patch_width+j-width_begin] = val

    return rebuild_img
                
rebuild_img = build_mask_taking_max(list_patches, 608, 608)

npt.assert_equal(rebuild_img[399,399],1)
npt.assert_equal(rebuild_img[0,607],1)
npt.assert_equal(rebuild_img[607,0],1)
npt.assert_equal(rebuild_img[607,607],1)